In [ ]:
# Entrenamiento del modelo MLP con 7 experimentos

In [ ]:
Grupo: 1

Estudiantes:

-Constanza Olivos Fernandez

-Javier Nanco Becerra

-Nicolás Pozo Villagrán

Fecha: 20-09-2025

Version: 1.0

In [ ]:
# Objetivos del notebook

In [ ]:
# Optimización de Hiperparámetros para Modelo MLP en Predicción de Rendimiento de Trigo

## Objetivo del Estudio
**Identificar la combinación óptima de hiperparámetros** (batch size y learning rate) para un modelo de Perceptrón Multicapa (MLP) mediante la ejecución sistemática de 7 experimentos controlados, con el fin de maximizar la precisión predictiva del rendimiento de cultivos de trigo.

## Objetivos Específicos

### 1. Evaluación Comparativa de Configuraciones
- Analizar el impacto de **5 valores de batch size** (16, 32, 64, 128, 256) en el desempeño del modelo
- Evaluar el efecto de **4 tasas de aprendizaje** (0.0001, 0.001, 0.002, 0.005, 0.01) en la convergencia y estabilidad
- Determinar la **combinación sinérgica** que produce el mejor resultado predictivo

### 2. Análisis de Comportamiento del Modelo
- Medir la **capacidad de generalización** mediante métricas de rendimiento (R², RMSE, MAE)
- Identificar patrones de **convergencia y estabilidad** durante el entrenamiento
- Detectar configuraciones propensas a **sobreajuste o subajuste**

### 3. Establecimiento de Línea Base
- Crear un **benchmark de desempeño** para futuras optimizaciones
- Documentar el **proceso experimental** para reproducibilidad
- Generar **recomendaciones prácticas** para configuraciones de MLP en problemas similares

## Hipótesis de Trabajo
"Una configuración de **batch size moderado (32-64)** combinada con un **learning rate relativamente alto (0.01)** producirá el mejor equilibrio entre velocidad de convergencia y capacidad predictiva final en modelos MLP para datos agronómicos."

## Métrica Principal de Evaluación
- **Variable principal**: Coeficiente de determinación (R²)
- **Variables secundarias**: RMSE, MAE, Pérdida final de entrenamiento
- **Criterio de selección**: Maximización de R² en conjunto de prueba

## Alcance del Análisis
El presente estudio se centra exclusivamente en la **optimización hiperparamétrica básica**, sentando las bases para investigaciones futuras que podrían incluir:
- Búsqueda más exhaustiva de hiperparámetros
- Optimización de arquitectura de red
- Técnicas avanzadas de regularización
- Validación en múltiples conjuntos de datos

## Valor Agregado
Este experimento proporciona **evidencia empírica** sobre la sensibilidad de los modelos MLP a configuraciones hiperparamétricas específicas en el dominio agronómico, facilitando decisiones informadas en futuros desarrollos de modelos predictivos.

In [1]:
# ===========================
# Entrenamiento del modelo MLP con 7 experimentos
# ===========================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
import os
from datetime import datetime

# Configurar TensorFlow para output mínimo
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# ===========================
# 1. Cargar y preparar datos
# ===========================
csv_path = os.path.join('csv', 'features_trigo.csv')
df = pd.read_csv(csv_path)
print(" Dataset cargado")

# One-hot encoding
df_encoded = pd.get_dummies(df, drop_first=True)

# Features y target
X = df_encoded.drop("Rendimiento_kg_ha", axis=1)
y = df_encoded["Rendimiento_kg_ha"]

# Split 70/15/15
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# ===========================
# 2. Crear DataFrame para registro de experimentos
# ===========================
experimentos_df = pd.DataFrame(columns=[
    'Experimento', 'Fecha', 'Batch_Size', 'Learning_Rate', 'Epochs_Final',
    'R2', 'RMSE', 'MAE', 'Loss_Final', 'Loss_Val_Final', 'Observaciones'
])

# ===========================
# 3. Función para crear modelo MLP con learning rate configurable
# ===========================
def crear_mlp(learning_rate=0.001):
    model = Sequential([
        Input(shape=(X_train_scaled.shape[1],)),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss='mse', metrics=['mse'])
    return model

# ===========================
# 4. Función para ejecutar y registrar experimentos
# ===========================
def ejecutar_experimento(nombre, batch_size, learning_rate, observaciones=""):
    print(f" Ejecutando {nombre}...")
    
    # Entrenar modelo
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
    
    model = crear_mlp(learning_rate=learning_rate)
    
    history = model.fit(
        X_train_scaled, y_train,
        validation_data=(X_valid_scaled, y_valid),
        epochs=100,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=0
    )
    
    # Evaluar modelo
    y_pred = model.predict(X_test_scaled, verbose=0).flatten()
    
    # Calcular métricas
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    final_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    epochs_final = len(history.history['loss'])
    
    # Registrar experimento
    nuevo_exp = {
        'Experimento': nombre,
        'Fecha': datetime.now().strftime("%Y-%m-%d %H:%M"),
        'Batch_Size': batch_size,
        'Learning_Rate': learning_rate,
        'Epochs_Final': epochs_final,
        'R2': round(r2, 4),
        'RMSE': round(rmse, 2),
        'MAE': round(mae, 2),
        'Loss_Final': round(final_loss, 2),
        'Loss_Val_Final': round(final_val_loss, 2),
        'Observaciones': observaciones
    }
    
    global experimentos_df
    experimentos_df = pd.concat([experimentos_df, pd.DataFrame([nuevo_exp])], ignore_index=True)
    
    print(f"✅ {nombre} completado - R²: {r2:.4f}, Loss: {final_loss:.2f}")
    
    return model, history

# ===========================
# 5. Ejecutar 7 experimentos diferentes
# ===========================
print(" Iniciando 7 experimentos...")

# Experimento 1: Configuración base
model1, history1 = ejecutar_experimento(
    nombre="Exp1",
    batch_size=32,
    learning_rate=0.001,
    observaciones="Configuración base, convergencia estable"
)

# Experimento 2: Learning rate más alto
model2, history2 = ejecutar_experimento(
    nombre="Exp2", 
    batch_size=32,
    learning_rate=0.01,
    observaciones="LR alto, posible inestabilidad"
)

# Experimento 3: Learning rate más bajo
model3, history3 = ejecutar_experimento(
    nombre="Exp3",
    batch_size=32, 
    learning_rate=0.0001,
    observaciones="LR bajo, convergencia lenta"
)

# Experimento 4: Batch size más grande
model4, history4 = ejecutar_experimento(
    nombre="Exp4",
    batch_size=128,
    learning_rate=0.001,
    observaciones="Batch grande, entrenamiento más rápido"
)

# Experimento 5: Batch size más pequeño
model5, history5 = ejecutar_experimento(
    nombre="Exp5",
    batch_size=16,
    learning_rate=0.001,
    observaciones="Batch pequeño, más updates por época"
)

# Experimento 6: Combinación óptima
model6, history6 = ejecutar_experimento(
    nombre="Exp6",
    batch_size=64,
    learning_rate=0.002,
    observaciones="Combinación balanceada"
)

# Experimento 7: Configuración agresiva
model7, history7 = ejecutar_experimento(
    nombre="Exp7",
    batch_size=256,
    learning_rate=0.005,
    observaciones="Configuración agresiva, riesgo de divergencia"
)

# ===========================
# 6. Mostrar y guardar resultados
# ===========================
print("\n" + "="*90)
print("REGISTRO DE 7 EXPERIMENTOS - MLP")
print("="*90)

# Mostrar tabla formateada
resultados_display = experimentos_df.copy()
resultados_display['Configuración'] = 'batch = ' + experimentos_df['Batch_Size'].astype(str) + ', lr = ' + experimentos_df['Learning_Rate'].astype(str)
resultados_display['Resultado'] = 'R²: ' + (experimentos_df['R2'] * 100).round(1).astype(str) + '%; Loss: ' + experimentos_df['Loss_Final'].round(1).astype(str) + '; RMSE: ' + experimentos_df['RMSE'].astype(str)

tabla_final = resultados_display[['Experimento', 'Configuración', 'Resultado', 'Observaciones']]
print(tabla_final.to_string(index=False))

# Guardar registro completo
experimentos_df.to_csv("csv/registro_experimentos_mlp.csv", index=False)
print(f"\nRegistro de 7 experimentos guardado en 'csv/registro_experimentos_mlp.csv'")

# ===========================
# 7. Identificar y guardar el mejor modelo
# ===========================
mejor_exp = experimentos_df.loc[experimentos_df['R2'].idxmax()]
mejores_modelos = [model1, model2, model3, model4, model5, model6, model7]
mejor_modelo = mejores_modelos[experimentos_df['R2'].idxmax()]

# Guardar el mejor modelo
mejor_modelo.save("modelos/MLP_mejor_modelo.h5")
print(f"MEJOR EXPERIMENTO: {mejor_exp['Experimento']}")
print(f"   Configuración: batch = {mejor_exp['Batch_Size']}, lr = {mejor_exp['Learning_Rate']}")
print(f"   R²: {mejor_exp['R2']:.4f}")
print(f"   RMSE: {mejor_exp['RMSE']:.2f}")
print(f"   MAE: {mejor_exp['MAE']:.2f}")
print(f"   Épocas: {mejor_exp['Epochs_Final']}")
print(f"   Observaciones: {mejor_exp['Observaciones']}")

# ===========================
# 8. Análisis comparativo
# ===========================
print("\nANÁLISIS COMPARATIVO:")
print(f"   Rango de R²: {experimentos_df['R2'].min():.4f} - {experimentos_df['R2'].max():.4f}")
print(f"   Mejor batch size: {mejor_exp['Batch_Size']}")
print(f"   Mejor learning rate: {mejor_exp['Learning_Rate']}")
print(f"   Diferencia entre mejor/peor R²: {(experimentos_df['R2'].max() - experimentos_df['R2'].min()):.4f}")

# Guardar métricas resumidas
resumen_metricas = {
    'Total_Experimentos': len(experimentos_df),
    'Mejor_R2': experimentos_df['R2'].max(),
    'Peor_R2': experimentos_df['R2'].min(),
    'R2_Promedio': experimentos_df['R2'].mean(),
    'Mejor_Configuracion': f"batch={mejor_exp['Batch_Size']}, lr={mejor_exp['Learning_Rate']}",
    'Mejor_Experimento': mejor_exp['Experimento']
}

pd.DataFrame([resumen_metricas]).to_csv("csv/resumen_experimentos_mlp.csv", index=False)
print("Resumen de métricas guardado en 'csv/resumen_experimentos_mlp.csv'")

✅ Dataset cargado
🚀 Iniciando 7 experimentos...
🧪 Ejecutando Exp1...


C:\Users\javie\AppData\Local\Temp\ipykernel_41672\2800762466.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experimentos_df = pd.concat([experimentos_df, pd.DataFrame([nuevo_exp])], ignore_index=True)


✅ Exp1 completado - R²: 0.7977, Loss: 336329.09
🧪 Ejecutando Exp2...
✅ Exp2 completado - R²: 0.8205, Loss: 236732.55
🧪 Ejecutando Exp3...
✅ Exp3 completado - R²: 0.7111, Loss: 390126.56
🧪 Ejecutando Exp4...
✅ Exp4 completado - R²: 0.7445, Loss: 376843.03
🧪 Ejecutando Exp5...
✅ Exp5 completado - R²: 0.7976, Loss: 361023.66
🧪 Ejecutando Exp6...
✅ Exp6 completado - R²: 0.7930, Loss: 366013.44
🧪 Ejecutando Exp7...


✅ Exp7 completado - R²: 0.7946, Loss: 326125.44

📊 REGISTRO DE 7 EXPERIMENTOS - MLP
Experimento           Configuración                               Resultado                                 Observaciones
       Exp1  batch = 32, lr = 0.001  R²: 79.8%; Loss: 336329.1; RMSE: 332.9      Configuración base, convergencia estable
       Exp2   batch = 32, lr = 0.01 R²: 82.0%; Loss: 236732.6; RMSE: 313.54                LR alto, posible inestabilidad
       Exp3 batch = 32, lr = 0.0001 R²: 71.1%; Loss: 390126.6; RMSE: 397.81                   LR bajo, convergencia lenta
       Exp4 batch = 128, lr = 0.001 R²: 74.4%; Loss: 376843.0; RMSE: 374.07        Batch grande, entrenamiento más rápido
       Exp5  batch = 16, lr = 0.001 R²: 79.8%; Loss: 361023.7; RMSE: 332.99          Batch pequeño, más updates por época
       Exp6  batch = 64, lr = 0.002 R²: 79.3%; Loss: 366013.4; RMSE: 336.72                        Combinación balanceada
       Exp7 batch = 256, lr = 0.005 R²: 79.5%; Loss: 326125.4;

# Análisis de Experimentos MLP

## 1. Resultados Destacados
- **Mejor experimento: Exp2**
  - Configuración: `batch = 32, lr = 0.01`
  - R² = **0.8205**
  - RMSE = **313.54**
  - MAE = **247.51**
  - Observación: un learning rate más alto permitió un mejor aprendizaje, aunque con riesgo de inestabilidad.

- **Peor experimento: Exp3**
  - Configuración: `batch = 32, lr = 0.0001`
  - R² = **0.7111**
  - RMSE = **397.81**
  - Observación: el learning rate demasiado bajo provocó convergencia lenta y bajo desempeño.

---

## 2. Impacto del Batch Size
- **Batch pequeño (16)** → buen desempeño (R² ≈ 0.798), pero entrenamiento más lento por mayor número de actualizaciones.  
- **Batch grande (256)** → desempeño aceptable (R² ≈ 0.795), pero con riesgo de saltarse mínimos locales.  
- **Batch intermedio (32 y 64)** → ofrecen un buen equilibrio entre estabilidad y velocidad de entrenamiento.  

---

## 3. Tendencias Identificadas
- El **learning rate** tiene mayor influencia que el batch size.  
  - Muy bajo → convergencia lenta y peores métricas.  
  - Muy alto → puede dar el mejor resultado, pero con riesgo de inestabilidad.  
- El **batch size** afecta más la **estabilidad y velocidad** que la métrica final.

---

## 4. Diferencia entre Mejor y Peor
- Mejor R²: **0.8205 (Exp2)**  
- Peor R²: **0.7111 (Exp3)**  
- Brecha de **0.1094 (~11%)** en capacidad de explicación → diferencia significativa en el rendimiento del modelo.  

---

## 5. Conclusión
- **Exp2 (batch = 32, lr = 0.01)** es la mejor configuración, aunque requiere vigilancia por posible inestabilidad.  
- **Exp1 (batch = 32, lr = 0.001)** es más estable, con R² solo ligeramente menor (0.798 vs 0.8205).  
- Para futuros experimentos se recomienda:  
  - Probar **learning rates intermedios** (0.005, 0.007).  
  - Usar **early stopping** para mejorar la convergencia de modelos con LR bajos. 